In [0]:
df = spark.read.format("delta").load("/mnt/silver/covid_combined")

In [0]:
df.createOrReplaceGlobalTempView("covidCombined")
spark.catalog.cacheTable("global_temp.covidCombined")

In [0]:
df_2 = spark.sql("""
          with cte as (
          select location,year,month,sum(covid_total_cases) as covid_total_cases, sum(covid_new_cases) as covid_new_cases,
          sum(covid_total_deaths) as covid_total_deaths ,sum(covid_new_deaths) as covid_new_deaths,sum(covid_total_tests) as covid_total_tests,
          sum(covid_new_tests) as covid_new_tests, sum(covid_total_vax) as covid_total_vax,sum(covid_people_vax) as covid_people_vax, 
          sum(covid_fully_vax) as covid_fully_vax
          from global_temp.covidCombined
          group by location,year,month)

          select *,dense_rank() over (partition by location,year order by covid_total_cases desc) as monthly_covid_case_rank,
          dense_rank() over (partition by location,year order by covid_new_cases) as monthly_covid_new_case_rank from cte
          order by location,year,monthly_covid_case_rank
          """)

In [0]:
df_2.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year","month") \
    .save("/mnt/gold/covid_monthly_summary")